# Problem Set 1 - Neural network implementation

As described in section "3 Neural network implementation" of assignment 1, the goal is to build a shallow neural network from scratch using different approaches. To validate that your code is working and that the network is actually learning something, please use the following MNIST classification task. Finally, please submit proof of the learning progress as described in the assignment.

## Imports

In [ ]:
import random
import pandas as pd
import numpy as np
from sklearn import model_selection
import sklearn.datasets as sk_datasets
import torchvision.datasets as torch_datasets
from torchvision import transforms
import torch
import matplotlib.pyplot as plt

from scratch.network import Network
from scratch.res_network import ResNetwork
from pytorch.network import TorchNetwork
from scratch.utils import *

In [3]:
# Automatically load changes in imported modules
%load_ext autoreload
%autoreload 2

# Explicitly set seed for reproducibility
GLOBAL_RANDOM_STATE = 42

random.seed(GLOBAL_RANDOM_STATE)
np.random.seed(GLOBAL_RANDOM_STATE)

## A) Neural Network Classifier from Scratch

### Data

In [4]:
# Download MNIST dataset
x, y_cat = sk_datasets.fetch_openml('mnist_784', version=1, return_X_y=True, cache=True, as_frame=False)

# Preprocess dataset
x = (x / 255).astype('float32')
y_cat = y_cat.astype(int)
# One-hot encode y
y = np.zeros((len(y_cat), 10))
for i, val in enumerate(y_cat):
    y[i, val] = 1

# Use only small subset of data for faster training
x = x[:1000]
y = y[:1000]

# Split data into train and validation set
x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y, test_size=0.2, random_state=GLOBAL_RANDOM_STATE)



### ML Model & Training

In [ ]:
fnn = Network(sizes=[784, 128, 64, 10], learning_rate=0.1, epochs=50)
fnn.fit(x_train, y_train, x_val, y_val, cosine_annealing_lr=False)

### Test cosine annealing scheduler

In [ ]:
fnn.fit(x_train, y_train, x_val, y_val, cosine_annealing_lr=True)

### Test residual neural network

In [ ]:
res_nn = ResNetwork(sizes=[784, 128, 128, 10], learning_rate=1, epochs=50)
res_nn.fit(x_train, y_train, x_val, y_val)

## B) Neural Network Classifier using Torch

### Data

In [48]:
# Define data preprocessing steps
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
            ])

# Download MNIST dataset
train_set = torch_datasets.MNIST('data', train=True, download=True, transform=transform)
val_set = torch_datasets.MNIST('data', train=False, download=True, transform=transform)

# Use only small subset of data for faster training
train_set = torch.utils.data.Subset(train_set, range(1000))
val_set = torch.utils.data.Subset(val_set, range(1000))

# Utilize PyTorch DataLoader from simplified & harmonized loading of data
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=1)


### ML Model & Training

In [ ]:
torch_nn = TorchNetwork(sizes=[784, 128, 64, 10], learning_rate=0.2, epochs=50, random_state=GLOBAL_RANDOM_STATE)
torch_nn.fit(train_loader, val_loader)

## C) Visualize accuracy & hyperparameter tuning

Here, you should compare the accuracy of all trained models. Optionally, you can also show the results of hyperparameter tuning and comment which hyperparameters work best for this task.

In [ ]:
### BEGIN SOLUTION #### Feedforward NNfnn = Network(sizes=[784, 128, 64, 10], learning_rate=0.1, epochs=50)fnn.fit(x_train, y_train, x_val, y_val)# Feedforward NN + Cosine Annealingfnn_cosine = Network(sizes=[784, 128, 64, 10], learning_rate=0.1, epochs=50)fnn_cosine.fit(x_train, y_train, x_val, y_val, cosine_annealing_lr=True)# Residual NNres_nn = ResNetwork(sizes=[784, 128, 128, 10], learning_rate=0.1, epochs=50)res_nn.fit(x_train, y_train, x_val, y_val)# PyTorch NNtorch_nn = TorchNetwork(sizes=[784, 128, 64, 10], learning_rate=0.2, epochs=50, random_state=GLOBAL_RANDOM_STATE)torch_nn.fit(train_loader, val_loader)# Compute accuracies for all the modelsfnn_train_acc = float(fnn.compute_accuracy(x_train, y_train))fnn_val_acc = float(fnn.compute_accuracy(x_val, y_val))fnn_cosine_train_acc = float(fnn_cosine.compute_accuracy(x_train, y_train))fnn_cosine_val_acc = float(fnn_cosine.compute_accuracy(x_val, y_val))res_nn_train_acc = float(res_nn.compute_accuracy(x_train, y_train))res_nn_val_acc = float(res_nn.compute_accuracy(x_val, y_val))torch_train_acc = torch_nn.predict(train_loader)torch_val_acc = torch_nn.predict(val_loader)# Build dictionaryresults = {    "Feedforward NN": (fnn_train_acc, fnn_val_acc),    "Feedforward + CosineAnnealing": (fnn_cosine_train_acc, fnn_cosine_val_acc),    "Residual NN": (res_nn_train_acc, res_nn_val_acc),    "Torch NN": (torch_train_acc, torch_val_acc)}# Plotmodels = list(results.keys())train_accs = [results[m][0] for m in models]val_accs = [results[m][1] for m in models]x = np.arange(len(models))width = 0.35plt.figure(figsize=(10,6))plt.bar(x - width/2, train_accs, width, label='Train')plt.bar(x + width/2, val_accs, width, label='Validation')plt.xticks(x, models, rotation=20)plt.ylabel("Accuracy")plt.title("MNIST Classification Accuracy")plt.legend()plt.grid(axis='y')plt.show()### END SOLUTION ###